# Stochastic Hill Climber

Using this search-based black-box test, we aim to automatically find adversarial samples for detecting potential model biases.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import onnxruntime as rt
import onnx
from skl2onnx.common.data_types import FloatTensorType
from skl2onnx import to_onnx
from sklearn.feature_selection import VarianceThreshold
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from skl2onnx import convert_sklearn
import seaborn as sns
import math

/var/folders/g5/pq1216cd2x310s210s2jx33w0000gn/T/ipykernel_81371/1786036269.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


### Load the synthetic dataset

In [4]:
# Let's load the dataset
data = pd.read_csv('data/synth_data_for_training.csv')

# Let's specify the features and the target
y = data['checked']
X = data.drop(['checked'], axis=1)
X = X.astype(np.float32)

# Let's split the dataset into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Print size of train and test
print(X_train.shape, X_test.shape)




def convert_dataframe_schema(df, drop=None):
    inputs = []
    for k, v in zip(df.columns, df.dtypes):
        if drop is not None and k in drop:
            continue

        t = FloatTensorType([None, 1])

        inputs.append((k, t))
    return inputs


initial_inputs = convert_dataframe_schema(X_train)


inputs = {c: X_test[c].values for c in X_test.columns}
for (c, _) in initial_inputs:
    inputs[c] = inputs[c].astype(np.float32)
for k in inputs:
    inputs[k] = inputs[k].reshape((inputs[k].shape[0], 1))


(9483, 315) (3162, 315)


### Load the model

In [6]:
# Let's load the model

new_session = rt.InferenceSession("model/model_2_external.onnx")

# Let's predict the target

y_pred_onnx2 =  new_session.run(None, inputs)

accuracy_onnx_model = accuracy_score(y_test, y_pred_onnx2[0])

print('Accuracy of the ONNX model: ', accuracy_onnx_model)

# Get model confidence for random test sample
model_confs_test = y_pred_onnx2[1]
print(f"{len(model_confs_test)} confidence pairs on test data: {model_confs_test}")

Accuracy of the ONNX model:  0.9522454142947502
3162 confidence pairs on test data: [{0: 0.9997839331626892, 1: 0.00021606683731079102}, {0: 0.040233492851257324, 1: 0.9597665071487427}, {0: 0.97184157371521, 1: 0.02815842628479004}, {0: 0.992871105670929, 1: 0.007128894329071045}, {0: 0.999998927116394, 1: 1.0728836059570312e-06}, {0: 0.9039708375930786, 1: 0.09602916240692139}, {0: 0.993920087814331, 1: 0.006079912185668945}, {0: 0.8892831802368164, 1: 0.1107168197631836}, {0: 0.9602437615394592, 1: 0.03975623846054077}, {0: 0.7274260520935059, 1: 0.27257394790649414}, {0: 0.964984118938446, 1: 0.035015881061553955}, {0: 0.9987016916275024, 1: 0.0012983083724975586}, {0: 0.9979323148727417, 1: 0.0020676851272583008}, {0: 0.9947441816329956, 1: 0.0052558183670043945}, {0: 0.9966256618499756, 1: 0.003374338150024414}, {0: 0.9999963045120239, 1: 3.6954879760742188e-06}, {0: 0.9996143579483032, 1: 0.00038564205169677734}, {0: 0.9837355613708496, 1: 0.01626443862915039}, {0: 0.99893039464